In [ ]:
!pip install --upgrade openpyxl

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

#Catalogos

Catalogo de productos



In [ ]:
ab=pd.read_excel("/content/drive/Shareddrives/Catalogos_SO/Prod_Holding.xlsx",engine="openpyxl")
ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2242 entries, 0 to 2241
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   key              2241 non-null   object
 1   Site             2242 non-null   object
 2   Holding          2242 non-null   object
 3   identificador    2129 non-null   object
 4   descripcion      2242 non-null   object
 5   CODIGO           2003 non-null   object
 6   codigo_anterior  53 non-null     object
 7   EAN              670 non-null    object
 8   Factor_Caja      2199 non-null   object
dtypes: object(9)
memory usage: 157.8+ KB


In [ ]:
# seleccion de columnas por index
ab=ab.iloc[:,[1,2,3,5]]

# filtrado sobre el catalogo
ab = ab[ab['Site'] == 'MTY']
ab = ab[ab['Holding'] == 'SUPER AKI']

# selccionar columnas por index
ab= ab.iloc[:,[2,3]]

# renombrar las columnas
ab= ab.rename(columns={'identificador': "CLAVE"})
ab= ab.rename(columns={'CODIGO': "AB actual"})

In [ ]:
ab.head()

,CLAVE,AB actual
1188,288937,AB19090074
1189,288939,AB12101521
1190,288940,AB12335678
1191,288943,AB16335100
1192,288945,AB10335223


Catalogo de tiendas

In [ ]:
tda= pd.read_excel("/content/drive/Shareddrives/Catalogos_SO/Tda Super Aki.xlsx", engine="openpyxl") # Explicitly use the openpyxl engine

tda= tda.iloc[:, [1,2]]

tda.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Código Punto de Venta Cadena,Nombre de Tienda
0,83,Aki Alameda
1,80,Aki Colosio
2,28,Aki Concordia
3,25,Aki Gm Villa Del Rio
4,78,Aki Av. Centenario


Arreglo de columnas en el catalogo de tiendas

In [ ]:
tda= tda.rename(columns={'Código Punto de Venta Cadena': "id"})  #renombrar una columna

tda = tda[["id", "Nombre de Tienda",]]  #reordenar las columnas

tda['Nombre de Tienda']=tda['Nombre de Tienda'].str.strip() #eliminar espacios en blanco en toda posicion

tda.head()

,id,Nombre de Tienda
0,83,Aki Alameda
1,80,Aki Colosio
2,28,Aki Concordia
3,25,Aki Gm Villa Del Rio
4,78,Aki Av. Centenario


#Semanal

##Venta

In [ ]:
#omitir notacion cientifica
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
sem = pd.read_excel("/content/drive/MyDrive/Sell Out/Super Aki/Mensual/01_25.xlsx")  # Use sep=None to auto-detect the delimiter

sem=sem.replace(r'\s+',' ', regex=True) # espacios en blanco

sem=sem.iloc[:, [0,1,10,11,13,17,18,19,20]]

sem.head()

,id,Sucursal,Unnamed: 10,Producto (S),Factor empaque (S),Día,Existencias (uni),Sell out (uni),Sell out ($)
0,100,AKI SAN ANTONIO CUCUL,327297,PAPEL HIGIÉNICO ELITE SOFTMAX,4,2025/01/06,NaN,1.00,104.31
1,100,AKI SAN ANTONIO CUCUL,327297,PAPEL HIGIÉNICO ELITE SOFTMAX,4,2025/01/07,NaN,1.00,104.31
2,100,AKI SAN ANTONIO CUCUL,327297,PAPEL HIGIÉNICO ELITE SOFTMAX,4,2025/01/08,NaN,1.00,104.31
3,100,AKI SAN ANTONIO CUCUL,327297,PAPEL HIGIÉNICO ELITE SOFTMAX,4,2025/01/17,NaN,3.00,312.93
4,100,AKI SAN ANTONIO CUCUL,327297,PAPEL HIGIÉNICO ELITE SOFTMAX,4,2025/01/20,NaN,1.00,104.31


##Conversiones

In [ ]:
sem["invt_cjas"]=sem['Existencias (uni)']/sem["Factor empaque (S)"]
sem["vtas_cjas"]=sem['Sell out (uni)']/sem["Factor empaque (S)"]

Transformar el tipo de dato de algunas columnas

In [ ]:
sem['Día'] = sem['Día'].astype('datetime64[ns]')

sem= sem.rename(columns={'Unnamed: 10': 'clave_prod'})  #renombrar una columna unnamed
sem= sem.rename(columns={'id': "determinante"})

Seleccionar el rango de fechas

In [ ]:
#rango de fechas
start_date = datetime.strptime('01/01/2025', '%m/%d/%Y')
end_date = datetime.strptime('01/31/2025', '%m/%d/%Y')

#filtrar el rango de fechas dadas
sem = sem[(sem['Día'] >= start_date) & (sem['Día'] <= end_date)]

valores perdidos

In [ ]:
print("Missing values in semanal:")
print(sem.isnull().sum())
print("")
sem.shape

Missing values in semanal:
determinante          0
Sucursal              0
clave_prod            0
Producto (S)          0
Factor empaque (S)    0
Día                   0
Existencias (uni)     0
Sell out (uni)        0
Sell out ($)          0
invt_cjas             0
vtas_cjas             0
dtype: int64



(8652, 11)

Sustituir valores negativos

In [ ]:
sem['invt_cjas'] = np.where(sem['invt_cjas'] < 0, 0, sem['invt_cjas'])
sem['vtas_cjas'] = np.where(sem['vtas_cjas'] < 0, 0, sem['vtas_cjas'])
sem['Sell out ($)']=np.where(sem['Sell out ($)'] < 0, 0, sem['Sell out ($)'])

Fill null values

In [ ]:
sem.fillna(0, inplace=True)

##Dividir las bases

**1) ventas**

In [ ]:
vtas=sem.iloc[:,[0,1,2,3,8,10]]

Seleccionar valores > 0

In [ ]:
vtas = vtas[(vtas['Sell out ($)']>0) & (vtas['vtas_cjas']>0)]

In [ ]:
vtas.head()

,determinante,Sucursal,clave_prod,Producto (S),Sell out ($),vtas_cjas
0,100,AKI SAN ANTONIO CUCUL,327297,PAPEL HIGIÉNICO ELITE SOFTMAX,104.31,0.25
1,100,AKI SAN ANTONIO CUCUL,327297,PAPEL HIGIÉNICO ELITE SOFTMAX,104.31,0.25
2,100,AKI SAN ANTONIO CUCUL,327297,PAPEL HIGIÉNICO ELITE SOFTMAX,104.31,0.25
3,100,AKI SAN ANTONIO CUCUL,327297,PAPEL HIGIÉNICO ELITE SOFTMAX,312.93,0.75
4,100,AKI SAN ANTONIO CUCUL,327297,PAPEL HIGIÉNICO ELITE SOFTMAX,104.31,0.25


**2) inventarios**

In [ ]:
#rango de fechas
start_date = datetime.strptime('01/31/2025', '%m/%d/%Y')
end_date = datetime.strptime('01/31/2025', '%m/%d/%Y')

#filtrar el rango de fechas dadas
invt = sem[(sem['Día'] >= start_date) & (sem['Día'] <= end_date)]

In [ ]:
invt=sem.iloc[:, [0,1,2,3,9]]

Seleccionar valores >0

In [ ]:
invt = invt[(invt['invt_cjas']>0) & (invt['invt_cjas']>0)]

In [ ]:
invt.tail()

,determinante,Sucursal,clave_prod,Producto (S),invt_cjas
8587,91,AKI GM RUTA 5,348693,PAPEL HIGIENICO BIG ROLL 600HD,9.00
8595,91,AKI GM RUTA 5,302515,SERVILLETA ELITE ULTRA,0.17
8610,91,AKI GM RUTA 5,327037,SERVILLETA ELITE ULTRA,1.54
8623,91,AKI GM RUTA 5,288943,SERVILLETA ELITE,1.08
8651,91,AKI GM RUTA 5,288945,TOALLA DE PAPEL PARA COCINA ELITE,5.20


### vtas + invt

In [ ]:
sos=pd.concat([vtas,invt])

sos.fillna(0, inplace=True)

ELIMINAR los siguientes centros de venta

In [ ]:
#eliminar los siguientes puntos de venta
condition =(sos['determinante'] == 31)

# Delete rows based on the condition
sos = sos[~condition]

REMPLAZAR los siguientes valores

In [ ]:
sos["determinante"] = sos["determinante"].replace([126], 127)

## Merged catalogos

In [ ]:
#productos
merged_sos= sos.merge(ab, left_on="clave_prod", right_on="CLAVE", how="left")

In [ ]:
#tda
merged_sos = merged_sos.merge(tda, left_on="determinante", right_on="id", how="left")

Observar valores faltantes en el dataset

In [ ]:
print(merged_sos.isnull().sum())

determinante        0
Sucursal            0
clave_prod          0
Producto (S)        0
Sell out ($)        0
vtas_cjas           0
invt_cjas           0
CLAVE               0
AB actual           0
id                  0
Nombre de Tienda    0
dtype: int64


In [ ]:
merged_sos[merged_sos.isna().any(axis=1)]  #revisar registros con datos faltantes

#merged_sos.dropna(inplace=True) #eliminar registros con datos faltantes

,determinante,Sucursal,clave_prod,Producto (S),Sell out ($),vtas_cjas,invt_cjas,CLAVE,AB actual,id,Nombre de Tienda


### Pivot Table

In [ ]:
#tabla dinamica
pivot_table=pd.pivot_table(data=merged_sos,
                           index=["id", "AB actual"],
                           values=["vtas_cjas", "Sell out ($)", "invt_cjas"],
                           aggfunc=np.sum,
                           fill_value=0)

pivot_table.reset_index().to_excel('sos.xlsx',sheet_name = 'Sheet1',index=False)

<ipython-input-44-6d61e1604b11>:2: FutureWarning: The provided callable <function sum at 0x79ca529eca40> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table=pd.pivot_table(data=merged_sos,
